## intro to OpenAI Agents SDK

In [11]:
from agents import Agent, Runner
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
agent = Agent(name="Assistant",model="o3-mini", instructions="Answer as factually as possible")
result = await Runner.run(agent, "Write a basic introduction about Telkomsel")
print(result.final_output)

Telkomsel is one of Indonesia's largest and most trusted telecommunications operators. Established in 1995 and primarily owned by Telkom Indonesia, the company offers a broad range of mobile services, including voice, SMS, data, and a growing suite of digital products and solutions. Known for its extensive network coverage, Telkomsel reaches both bustling urban centers and remote areas across the Indonesian archipelago, ensuring connectivity for millions of users. The company has continually invested in its infrastructure—upgrading from early technologies to 4G LTE and expanding into 5G—thus playing a pivotal role in Indonesia’s digital transformation and economic development.


In [10]:
from agents import Agent, Runner, function_tool
#function_tool: decorator to register tools
@function_tool
def get_weather(city: str) -> str:
    return f"Sunny in {city}"

#First Create an instance of Agent
agent = Agent(
    name="WeatherBot",
    instructions="Provide weather info",
    tools=[get_weather]
)
# in CLI: result = Runner.run_sync(agent, "What's the weather in Paris?") in python cli
# Call runner.run() to run the agent
result = await Runner.run(agent, "What's the weather in Paris?")
print(result.final_output)

The weather in Paris is currently sunny.


- Use a real weather API (e.g., Open-Meteo)
- Add input/output guardrails using pydantic

In [5]:
#!pip install httpx

In [9]:
# Step 1: Install dependencies if needed
# !pip install openai-agents httpx

# Step 2: Import required modules
import httpx
from agents import Agent, Runner, function_tool, trace

# Step 3: Define the weather tool
@function_tool
async def get_weather(city: str) -> str:
    try:
        # Get coordinates for the city
        geo = httpx.get("https://geocoding-api.open-meteo.com/v1/search", params={"name": city}).json()
        if "results" not in geo or not geo["results"]:
            return "City not found."

        lat = geo["results"][0]["latitude"]
        lon = geo["results"][0]["longitude"]

        # Fetch current weather data
        weather = httpx.get("https://api.open-meteo.com/v1/forecast", params={
            "latitude": lat,
            "longitude": lon,
            "current_weather": True
        }).json()

        temp = weather["current_weather"]["temperature"]
        return f"It is currently {temp}°C in {city}."
    except Exception as e:
        return f"Failed to get weather for {city}: {e}"

# Step 4: Define fallback agent
fallback_agent = Agent(
    name="FallbackAgent",
    instructions="Apologize if the weather data is not available or city is invalid."
)

# Step 5: Define main agent
main_agent = Agent(
    name="WeatherBot",
    instructions="You're a weather assistant. Use tools to answer weather questions. If city is unknown, hand off.",
    tools=[get_weather],
    handoffs={"unknown": fallback_agent}
)

# Step 6: Run agent with tracing (for Jupyter / async environments)
with trace(workflow_name="weather_query"):
    result = await Runner.run(main_agent, "What's the weather in Jakarta?")
    print(result.final_output)


It's currently 30.5°C in Jakarta.


In [24]:
from agents import Agent, WebSearchTool, Runner

In [29]:
async def main():
    # 1. Create an agent equipped with web search
    research_agent = Agent(
        name="Research Agent",
        instructions="You're a research assistant that uses web search to find up-to-date info and cite sources.",
        tools=[WebSearchTool()]
    )

    # 2. Run a query
    query = "Siapa direktur utama telkom indonesia?"
    result = await Runner.run(research_agent, query)

    # 3. Print the full final output with citations
    print("\n🔎 Final Answer:\n")
    print(result.final_output)

await main()


🔎 Final Answer:

Pada 27 Mei 2025, Rapat Umum Pemegang Saham Tahunan (RUPST) PT Telkom Indonesia (Persero) Tbk mengangkat Dian Siswarini sebagai Direktur Utama, menggantikan Ririek Adriansyah yang telah menjabat sejak 2019. ([antaranews.com](https://www.antaranews.com/berita/4862029/rupst-telkom-angkat-dian-siswarini-jadi-dirut/?utm_source=openai))

Sebelumnya, Dian Siswarini menjabat sebagai Presiden Direktur dan CEO PT XL Axiata Tbk sejak 2015 hingga mengundurkan diri pada Desember 2024. ([telko.id](https://telko.id/operator/telkom/telkom-tunjuk-dian-siswarini-gantikan-ririek-adriansyah-sebagai-dirut/?utm_source=openai))

Dengan pengalaman lebih dari dua dekade di industri telekomunikasi, termasuk transformasi digital di XL Axiata, Dian diharapkan dapat membawa Telkom Indonesia ke arah transformasi digital yang lebih progresif dan adaptif terhadap dinamika pasar yang terus berkembang. ([eksekutor.com](https://eksekutor.com/2025/05/27/dian-siswarini-ditunjuk-sebagai-direktur-utama-te

In [33]:
async def main():
    # 1. Create an agent equipped with web search
    research_agent = Agent(
        name="Research Agent",
        instructions="You are an assistant that returns structured tables based on user queries.",
        tools=[WebSearchTool()]
    )

    # 2. Run a query
    query = "Find internet providers in Madiun (if possible 6 unique provider name)  and return a table with columns: Provider Name, Package, Speed, Price."
    result = await Runner.run(research_agent, query)

    # 3. Print the full final output with citations
    print("\n🔎 Final Answer:\n")
    print(result.final_output)

await main()


🔎 Final Answer:

Berikut adalah daftar beberapa penyedia layanan internet di Madiun beserta paket, kecepatan, dan harga yang mereka tawarkan:

| Nama Penyedia | Paket           | Kecepatan       | Harga          |
|---------------|-----------------|-----------------|----------------|
| **Biznet**    | Home 0D         | Hingga 50 Mbps  | Rp 175.000/bulan |
|               | Home 1D         | Hingga 150 Mbps | Rp 375.000/bulan |
|               | Home 2D         | Hingga 250 Mbps | Rp 575.000/bulan |
|               | Gamers 3D       | Hingga 300 Mbps | Rp 700.000/bulan |
| **IndiHome**  | 10 Mbps         | 10 Mbps         | Rp 150.000/bulan |
|               | 20 Mbps         | 20 Mbps         | Rp 200.000/bulan |
|               | 50 Mbps         | 50 Mbps         | Rp 300.000/bulan |
|               | 100 Mbps        | 100 Mbps        | Rp 500.000/bulan |
| **MyRepublic**| Nova            | 30 Mbps         | Rp 329.000/bulan |
|               | Fast            | 50 Mbps         | Rp 